In [49]:
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import scipy.signal
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [50]:
def find_period(signal):
    acf = np.correlate(signal, signal, 'full')[-len(signal):]
    inflection = np.diff(np.sign(np.diff(acf)))
    peaks = (inflection < 0).nonzero()[0] + 1
    return peaks[acf[peaks].argmax()]

In [51]:
def make_seasonal_model(data, column):
    data["x"] = data[column].astype(float)
    data_array = data["x"].to_numpy()
    # state_means = data_array
    kf = KalmanFilter(transition_matrices=[1],
                    observation_matrices=[1],
                    initial_state_mean=data_array[0],
                    initial_state_covariance=1,
                    observation_covariance=5,
                    transition_covariance=1) #0.01) 
    state_means, state_covariances = kf.filter(data_array) 

    period = find_period(np.diff(state_means.flatten()))

    print(period)
    train_data = state_means.flatten()
    normal_order = (0,1,0)
    season_order = (1,0,1, period)
    model = SARIMAX(train_data, order= normal_order, seasonal_order = season_order)
    model_fit = model.fit()
    print(model_fit.summary())
    return model, model_fit

In [52]:
def make_prediction(model_fit, look_ahead):
    predictions = model_fit.forecast(look_ahead)
    return predictions

In [53]:
def build_linear_model_data(batches):
    load_data = pd.read_csv("./actual_load.csv")
    gen_data = pd.read_csv("./actual_generation.csv")
    label_data = pd.read_csv("./dap_outfile.csv")
    label_data = label_data['0'].astype(float)
    initial = 500
    
    load_predictions = []
    solar_predictions = []
    hydro_predictions = []
    biomass_predictions = []
    labels = []

    # how about trying to start this backwards

    for initial in range(501, batches*500+2, 24):
        load = load_data.iloc[initial-500:initial]
        gen = gen_data.iloc[initial-500:initial]
        label = label_data.iloc[initial:initial+24] 

        load_model, load_model_fit = make_seasonal_model(load, "Actual Load")
        solar_model, solar_model_fit = make_seasonal_model(gen, "Solar")
        hydro_model, hydro_model_fit = make_seasonal_model(gen, "Hydro Pumped Storage")
        biomass_model, biomass_model_fit = make_seasonal_model(gen, "Biomass")

    
        load_predictions.extend(make_prediction(load_model_fit, 24))
        solar_predictions.extend(make_prediction(solar_model_fit, 24))
        hydro_predictions.extend(make_prediction(hydro_model_fit, 24))
        biomass_predictions.extend(make_prediction(biomass_model_fit, 24))
        labels.extend(label)
    features = pd.DataFrame({"load" : load_predictions, "solar" : solar_predictions, "hydro" : hydro_predictions, "biomass" : biomass_predictions})
    labels = pd.DataFrame({"price" : labels})    
    return features, labels



In [54]:
# features, labels = build_linear_model_data(1)
# print(features)
# print(labels)
# reg = LinearRegression().fit(features, labels)
# print(reg.score(features, labels))

In [55]:
load_data = pd.read_csv("./actual_load.csv")
load_data = load_data.iloc[-200:-24]
load_model, load_model_fit = make_seasonal_model(load_data, "Actual Load")

gen_data = pd.read_csv("./actual_generation.csv")
gen_data = gen_data.iloc[-200:-24]
solar_model, solar_model_fit = make_seasonal_model(gen_data, "Solar")

hydro_model, hydro_model_fit = make_seasonal_model(gen_data, "Hydro Pumped Storage")

biomass_model, biomass_model_fit = make_seasonal_model(gen_data, "Biomass")


96
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.51681D+00    |proj g|=  3.32433D-01


/home/collin/.local/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.



At iterate    5    f=  7.49554D+00    |proj g|=  3.63339D-03

At iterate   10    f=  7.49538D+00    |proj g|=  9.78675D-03

At iterate   15    f=  7.49516D+00    |proj g|=  6.14980D-03

At iterate   20    f=  7.49308D+00    |proj g|=  1.40704D-02

At iterate   25    f=  7.49093D+00    |proj g|=  2.20852D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     27     35      1     0     0   1.416D-05   7.491D+00
  F =   7.4909311929271576     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
                                      SARIMAX Results                                       
Dep. Variable:      

/tmp/ipykernel_4853/1255016568.py:5: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  gen_data = pd.read_csv("./actual_generation.csv")
/home/collin/.local/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.33740D+00    |proj g|=  4.65543D-01

At iterate    5    f=  7.18296D+00    |proj g|=  3.39062D-02

At iterate   10    f=  7.17776D+00    |proj g|=  4.10501D-03

At iterate   15    f=  7.17311D+00    |proj g|=  7.12773D-02

At iterate   20    f=  6.88604D+00    |proj g|=  7.71885D-02

At iterate   25    f=  6.87708D+00    |proj g|=  2.68976D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     25     32      1     0     0   

/home/collin/.local/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.



At iterate    5    f=  6.51321D+00    |proj g|=  1.26590D-03

At iterate   10    f=  6.51261D+00    |proj g|=  1.58364D-02

At iterate   15    f=  6.49820D+00    |proj g|=  7.56469D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     19     23      1     0     0   3.213D-06   6.492D+00
  F =   6.4915749190377401     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
                                      SARIMAX Results                                       
Dep. Variable:                                    y   No. Observations:                  176
Model:             SARIMAX(0, 1, 0)x(1, 0, [1], 96)

/home/collin/.local/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.



At iterate    5    f=  3.47880D+00    |proj g|=  3.04134D-02

At iterate   10    f=  3.44676D+00    |proj g|=  4.31873D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     11     14      1     0     0   7.368D-06   3.447D+00
  F =   3.4467564946387586     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
                                      SARIMAX Results                                       
Dep. Variable:                                    y   No. Observations:                  176
Model:             SARIMAX(0, 1, 0)x(1, 0, [1], 96)   Log Likelihood                -606.629
Date:               

In [56]:
load_predictions = make_prediction(load_model_fit, 24)
solar_predictions = make_prediction(solar_model_fit, 24)
hydro_predictions = make_prediction(hydro_model_fit, 24)
biomass_predictions = make_prediction(biomass_model_fit, 24)

features = pd.DataFrame({"load" : load_predictions, "solar" : solar_predictions, "hydro" : hydro_predictions, "biomass" : biomass_predictions})
labels = pd.read_csv("./dap_outfile.csv")
labels = labels['0'].astype(float)
labels = labels.iloc[-24:]

reg = LinearRegression().fit(features, labels)
print(reg.score(features, labels))




0.8872236321507166
